In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import *
import pandas as pd

In [2]:
spark = SparkSession\
        .builder\
        .master("local")\
        .getOrCreate()

Let's Start by analyzing the boston neighbourhoods

In [3]:
boston_neighbourhoods_df = spark.read.csv("data/Boston/2020/June/neighbourhoods.csv", header=True)
boston_neighbourhoods_df.toPandas()

,neighbourhood_group,neighbourhood
0,None,Allston
1,None,Back Bay
2,None,Bay Village
3,None,Beacon Hill
4,None,Brighton
5,None,Charlestown
6,None,Chinatown
7,None,Dorchester
8,None,Downtown
9,None,East Boston


We can see that for Boston atleast, there are no neighbourhood_groups, so lets drop the column

In [4]:
boston_neighbourhoods_df = boston_neighbourhoods_df.drop('neighbourhood_group')
boston_neighbourhoods_df.toPandas()

,neighbourhood
0,Allston
1,Back Bay
2,Bay Village
3,Beacon Hill
4,Brighton
5,Charlestown
6,Chinatown
7,Dorchester
8,Downtown
9,East Boston


We will be running the analysis only on the top neighbourhoods that we selected from "" ,which had the highest YoY percent change.

In [5]:
neighbourhoods_list = ["Back Bay", 'South Boston', 'South End', 'Fenway', 'Allston', 'Dorchester', 'Downtown']
boston_neighbourhoods_df = spark.createDataFrame(neighbourhoods_list, StringType())
boston_neighbourhoods_df = boston_neighbourhoods_df.selectExpr("value as neighbourhood")
boston_neighbourhoods_df.toPandas()

,neighbourhood
0,Back Bay
1,South Boston
2,South End
3,Fenway
4,Allston
5,Dorchester
6,Downtown


Let's import Boston listings data

In [6]:
boston_listings_df_2020 = spark.read.csv("data/Boston/2020/June/listings_original.csv", header=True)
boston_listings_df_2020

DataFrame[id: string, name: string, host_id: string, host_name: string, neighbourhood_group: string, neighbourhood: string, latitude: string, longitude: string, room_type: string, price: string, minimum_nights: string, number_of_reviews: string, last_review: string, reviews_per_month: string, calculated_host_listings_count: string, availability_365: string]

Let's take a look at the neighbourhood data for listings

In [7]:
boston_listings_df_2020.limit(10).toPandas()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,3781,HARBORSIDE-Walk to subway,4804,Frank,None,East Boston,42.36413,-71.02991,Entire home/apt,125,28,16,2019-12-21,0.27,1,97
1,5506,**$49 Special ** Private! Minutes to center!,8229,Terry,None,Roxbury,42.32981,-71.09559,Entire home/apt,145,3,107,2020-05-01,0.78,6,333
2,6695,$99 Special!! Home Away! Condo,8229,Terry,None,Roxbury,42.32994,-71.09351,Entire home/apt,169,3,115,2019-11-02,0.87,6,317
3,8789,Curved Glass Studio/1bd facing Park,26988,Anne,None,Downtown,42.35919,-71.06265,Entire home/apt,99,91,25,2020-04-15,0.35,8,365
4,10730,Bright 1bed facing Golden Dome,26988,Anne,None,Downtown,42.3584,-71.06185,Entire home/apt,150,91,32,2020-04-16,0.25,8,282
5,10813,"Back Bay Apt-blocks to subway, Newbury St, The...",38997,Michelle,None,Back Bay,42.34961,-71.08904,Entire home/apt,150,28,80,2020-03-08,2.57,11,0
6,10986,Waterfront/North End Furnished Large Studio Apt,38997,Michelle,None,North End,42.36352,-71.05075,Entire home/apt,150,28,2,2016-05-23,0.04,11,364
7,16384,Small Room in Cambridge Kendall MIT,23078,Eric,None,Beacon Hill,42.3581,-71.07132,Private room,50,91,0,None,None,1,365
8,18711,The Dorset Redline | 3BR 1BA | Walk to Redline...,71783,Lance,None,Dorchester,42.32212,-71.06096,Entire home/apt,128,32,52,2019-10-05,0.42,39,330
9,22195,Copley House - Premium Studio,85130,Copley,None,Back Bay,42.34558,-71.0793,Private room,115,1,21,2020-02-08,0.17,6,286


We can drop neighbourhood_group from this dataframe as well

In [8]:
boston_listings_df_2020 = boston_listings_df_2020.drop('neighbourhood_group')


In [9]:
boston_listings_df_2020.limit(10).toPandas()

,id,name,host_id,host_name,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,3781,HARBORSIDE-Walk to subway,4804,Frank,East Boston,42.36413,-71.02991,Entire home/apt,125,28,16,2019-12-21,0.27,1,97
1,5506,**$49 Special ** Private! Minutes to center!,8229,Terry,Roxbury,42.32981,-71.09559,Entire home/apt,145,3,107,2020-05-01,0.78,6,333
2,6695,$99 Special!! Home Away! Condo,8229,Terry,Roxbury,42.32994,-71.09351,Entire home/apt,169,3,115,2019-11-02,0.87,6,317
3,8789,Curved Glass Studio/1bd facing Park,26988,Anne,Downtown,42.35919,-71.06265,Entire home/apt,99,91,25,2020-04-15,0.35,8,365
4,10730,Bright 1bed facing Golden Dome,26988,Anne,Downtown,42.3584,-71.06185,Entire home/apt,150,91,32,2020-04-16,0.25,8,282
5,10813,"Back Bay Apt-blocks to subway, Newbury St, The...",38997,Michelle,Back Bay,42.34961,-71.08904,Entire home/apt,150,28,80,2020-03-08,2.57,11,0
6,10986,Waterfront/North End Furnished Large Studio Apt,38997,Michelle,North End,42.36352,-71.05075,Entire home/apt,150,28,2,2016-05-23,0.04,11,364
7,16384,Small Room in Cambridge Kendall MIT,23078,Eric,Beacon Hill,42.3581,-71.07132,Private room,50,91,0,None,None,1,365
8,18711,The Dorset Redline | 3BR 1BA | Walk to Redline...,71783,Lance,Dorchester,42.32212,-71.06096,Entire home/apt,128,32,52,2019-10-05,0.42,39,330
9,22195,Copley House - Premium Studio,85130,Copley,Back Bay,42.34558,-71.0793,Private room,115,1,21,2020-02-08,0.17,6,286


In [10]:
boston_listings_df_2020.count()

3446

In [11]:
boston_listings_df_2020.select('neighbourhood').distinct().collect()

[Row(neighbourhood='Brighton'),
 Row(neighbourhood='42.31804'),
 Row(neighbourhood='North End'),
 Row(neighbourhood='Roxbury'),
 Row(neighbourhood=None),
 Row(neighbourhood='South End'),
 Row(neighbourhood='42.34786'),
 Row(neighbourhood='Roslindale'),
 Row(neighbourhood='42.28023'),
 Row(neighbourhood='Allston'),
 Row(neighbourhood='Charlestown'),
 Row(neighbourhood='Back Bay'),
 Row(neighbourhood='Mission Hill'),
 Row(neighbourhood='Mattapan'),
 Row(neighbourhood='Chinatown'),
 Row(neighbourhood='South Boston Waterfront'),
 Row(neighbourhood='Dorchester'),
 Row(neighbourhood='Longwood Medical Area'),
 Row(neighbourhood='Fenway'),
 Row(neighbourhood='42.36666'),
 Row(neighbourhood='Hyde Park'),
 Row(neighbourhood='Downtown'),
 Row(neighbourhood='West End'),
 Row(neighbourhood='Beacon Hill'),
 Row(neighbourhood='Bay Village'),
 Row(neighbourhood='Leather District'),
 Row(neighbourhood='South Boston'),
 Row(neighbourhood='West Roxbury'),
 Row(neighbourhood='42.28036'),
 Row(neighbourhoo

Little more cleanup is required to remove garbage values from neighbourhood columns

In [12]:
boston_listings_df_2020.createOrReplaceTempView('boston_listings_2020')

In [13]:
boston_neighbourhoods_df.createOrReplaceTempView('boston_neighbourhoods')

In [14]:
boston_listings_df_2020 = spark.sql("""
    SELECT * FROM boston_listings_2020 L
    WHERE L.neighbourhood IN 
    (SELECT neighbourhood FROM boston_neighbourhoods)
""")

In [15]:
boston_listings_df_2020.select('neighbourhood').distinct().collect()

[Row(neighbourhood='South End'),
 Row(neighbourhood='Allston'),
 Row(neighbourhood='Back Bay'),
 Row(neighbourhood='Dorchester'),
 Row(neighbourhood='Fenway'),
 Row(neighbourhood='Downtown'),
 Row(neighbourhood='South Boston')]

In [16]:
boston_listings_df_2020.count()

1667

We are going to compare how the increase in number of listings in a neighbourhood over the years might have contributed to increase in average rent in that neighbourhood

In [17]:
boston_listings_df_2020.createOrReplaceTempView('boston_listings_2020')

In [18]:
spark.sql("""
    SELECT neighbourhood, count(neighbourhood) as number_of_listings, avg(price) as average_price
    FROM boston_listings_2020
    GROUP BY neighbourhood
    ORDER BY number_of_listings desc
""").toPandas()

,neighbourhood,number_of_listings,average_price
0,Dorchester,420,159.152381
1,Downtown,277,304.444043
2,Back Bay,224,259.625000
3,South End,214,171.205607
4,Allston,197,107.497462
5,South Boston,168,191.321429
6,Fenway,167,224.365269


Let's perform similar analysis and get neighbourhood to number_of_listings ration for previous years

2019:

Read the data and Get the required neighbourhoods

In [19]:
boston_listings_df_2019 = spark.read.csv("data/Boston/2019/June/listings.csv", header=True)
boston_listings_df_2019.createOrReplaceTempView('boston_listings_2019')
boston_listings_df_2019 = spark.sql("""
    SELECT * FROM boston_listings_2019 L
    WHERE L.neighbourhood IN 
    (SELECT neighbourhood FROM boston_neighbourhoods)
""")

In [20]:
boston_listings_df_2019.count()

3002

Calculate number of listings and average price for each neighbourhood

In [21]:
boston_listings_df_2019.createOrReplaceTempView('boston_listings_2019')
spark.sql("""
    SELECT neighbourhood, count(neighbourhood) as number_of_listings, avg(price) as average_price
    FROM boston_listings_2019
    GROUP BY neighbourhood
    ORDER BY number_of_listings desc
""").toPandas()

,neighbourhood,number_of_listings,average_price
0,Dorchester,558,107.039427
1,Downtown,498,264.052209
2,Back Bay,471,332.592357
3,South End,404,227.284653
4,Fenway,400,235.577500
5,Allston,338,181.630178
6,South Boston,333,225.684685


2018:

Read the data and Get the required neighbourhoods

In [22]:
boston_listings_df_2018 = spark.read.csv("data/Boston/2018/July/listings.csv", header=True)
boston_listings_df_2018.createOrReplaceTempView('boston_listings_2018')
boston_listings_df_2018 = spark.sql("""
    SELECT * FROM boston_listings_2018 L
    WHERE L.neighbourhood IN 
    (SELECT neighbourhood FROM boston_neighbourhoods)
""")

In [23]:
boston_listings_df_2018.count()

2953

Calculate number of listings and average price for each neighbourhood

In [24]:
boston_listings_df_2018.createOrReplaceTempView('boston_listings_2018')
spark.sql("""
    SELECT neighbourhood, count(neighbourhood) as number_of_listings, avg(price) as average_price
    FROM boston_listings_2018
    GROUP BY neighbourhood
    ORDER BY number_of_listings desc
""").toPandas()

,neighbourhood,number_of_listings,average_price
0,Dorchester,519,99.107900
1,Fenway,498,237.586345
2,Back Bay,464,248.840517
3,South End,406,212.256158
4,Allston,393,156.821883
5,Downtown,361,252.728532
6,South Boston,312,211.294872


2017:

Read the data and Get the required neighbourhoods

In [25]:
boston_listings_df_2017 = spark.read.csv("data/Boston/2017/October/listings.csv", header=True)
boston_listings_df_2017.createOrReplaceTempView('boston_listings_2017')
boston_listings_df_2017 = spark.sql("""
    SELECT * FROM boston_listings_2017 L
    WHERE L.neighbourhood IN 
    (SELECT neighbourhood FROM boston_neighbourhoods)
""")

In [27]:
boston_listings_df_2017.count()

2375

Calculate number of listings and average price for each neighbourhood

In [29]:
boston_listings_df_2017.createOrReplaceTempView('boston_listings_2017')
spark.sql("""
    SELECT neighbourhood, count(neighbourhood) as number_of_listings, avg(price) as average_price
    FROM boston_listings_2017
    GROUP BY neighbourhood
    ORDER BY number_of_listings desc
""").toPandas()

,neighbourhood,number_of_listings,average_price
0,Back Bay,410,239.880488
1,Dorchester,398,98.256281
2,Fenway,357,210.862745
3,South End,354,214.519774
4,Allston,297,99.632997
5,Downtown,295,252.318644
6,South Boston,264,197.825758


2016:

Read the data and Get the required neighbourhoods

In [30]:
boston_listings_df_2016 = spark.read.csv("data/Boston/2016/September/listings.csv", header=True)
boston_listings_df_2016.createOrReplaceTempView('boston_listings_2016')
boston_listings_df_2016 = spark.sql("""
    SELECT * FROM boston_listings_2016 L
    WHERE L.neighbourhood IN 
    (SELECT neighbourhood FROM boston_neighbourhoods)
""")

In [31]:
boston_listings_df_2016.count()

1793

Calculate number of listings and average price for each neighbourhood

In [32]:
boston_listings_df_2016.createOrReplaceTempView('boston_listings_2016')
spark.sql("""
    SELECT neighbourhood, count(neighbourhood) as number_of_listings, avg(price) as average_price
    FROM boston_listings_2016
    GROUP BY neighbourhood
    ORDER BY number_of_listings desc
""").toPandas()

,neighbourhood,number_of_listings,average_price
0,South End,326,204.349693
1,Back Bay,302,240.950331
2,Fenway,290,220.393103
3,Dorchester,269,91.639405
4,Allston,260,112.307692
5,South Boston,174,187.609195
6,Downtown,172,236.459302


2015:

Read the data and Get the required neighbourhoods

In [33]:
boston_listings_df_2015 = spark.read.csv("data/Boston/2015/October/listings.csv", header=True)
boston_listings_df_2015.createOrReplaceTempView('boston_listings_2015')
boston_listings_df_2015 = spark.sql("""
    SELECT * FROM boston_listings_2015 L
    WHERE L.neighbourhood IN 
    (SELECT neighbourhood FROM boston_neighbourhoods)
""")

In [34]:
boston_listings_df_2015.count()

1248

In [35]:
boston_listings_df_2015.createOrReplaceTempView('boston_listings_2015')
spark.sql("""
    SELECT neighbourhood, count(neighbourhood) as number_of_listings, avg(price) as average_price
    FROM boston_listings_2015
    GROUP BY neighbourhood
    ORDER BY number_of_listings desc
""").toPandas()

,neighbourhood,number_of_listings,average_price
0,South End,251,216.964143
1,Allston,223,128.905830
2,Back Bay,206,248.800971
3,Fenway,185,249.908108
4,Dorchester,166,104.138554
5,South Boston,114,191.622807
6,Downtown,103,243.815534
